In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import tensorflow as tf
import edward as ed
from edward.models import Bernoulli, Categorical, Normal, Empirical, Multinomial

from utils.utils import load_dataframe, preprocess, transition_matrix

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)
sns.set_style('whitegrid')

## Data

In [3]:
df = load_dataframe()

Loading raw data from hdf5 cache...
Fetching raw data took 2.85 seconds
id             int64   
loan_status    category
age_of_loan    int64   
term           category
dtype: object
Retrieved 40,268,594 rows, 4 columns


In [4]:
# note I made this filter on term == 36 and age_of_loan <= 36 
df = preprocess(df)

Mapping transformations...
Preprocessing data...
Caching...
Preprocessing and caching took 129.73 seconds
id             int64
loan_status    int64
age_of_loan    int64
term           int64
dtype: object
Preprocessed 27,641,460 rows, 4 columns


In [9]:
realized_transitions = transition_matrix(df)

Loading transition matrix from hdf5 cache...
Fetching transition matrix took 0.04 seconds


In [10]:
realized_transitions

loan_status_y,Charged Off,Current,Default,Fully Paid,In Grace Period,Issued,Late (16-30 days),Late (31-120 days)
loan_status_x,,,,,,,,
Charged Off,0,0,0,0,0,0,0,0
Current,774,24453702,3,707322,5831,0,160366,62102
Default,28897,147,2297,71,0,0,4,506
Fully Paid,0,0,0,8063,12,0,101,72
In Grace Period,0,276,0,11,22,0,59,41
Issued,0,17206,0,670,1,0,38,1
Late (16-30 days),4548,32376,0,2066,257,0,13413,119621
Late (31-120 days),105934,25434,29802,2146,56,0,3292,332762


## Model

In [13]:
# from issue https://github.com/blei-lab/edward/issues/450
chain_len = 36
n_states = df.loan_status.unique().shape[0]

# create default starting state probability vector
x_0 = Categorical(probs=tf.fill([n_states], 1.0 / n_states))

# transition matrix
T = tf.nn.softmax(tf.Variable(tf.random_uniform([n_states, n_states])), axis=0)

# MODEL
x = []
for _ in range(chain_len):
    x_tm1 = x[-1] if x else x_0
    x_t = Categorical(probs=T[:, x_tm1])
    x.append(x_t)

## Inference

In [20]:
df.dtypes

id                int64
loan_status       int64
age_of_loan       int64
term              int64
previous_month    int64
dtype: object

In [36]:
df.loc[df.id == 55742].loan_status.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3], dtype=int64)

In [ ]:
qx = [Categorical(probs=tf.nn.softmax(tf.Variable(tf.ones(n_states)))) for _ in range(chain_len)]

x_data = df.loc[df.id == 55742].loan_status.values

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(T))

    inference = ed.KLqp(dict(zip(x, qx)), dict(zip(x, x_data)))
    inference.run(n_iter=20000)

    print(sess.run(T))
    print('qx:')
    pprint(sess.run([foo.probs for foo in qx]))
    print('x:')
    pprint(sess.run([foo.probs for foo in x]))

[[0.18844862 0.09978226 0.09664977 0.09886406 0.13895456 0.10338021
  0.10773823 0.13661277]
 [0.14108378 0.08936589 0.15624593 0.0896719  0.15368576 0.10087609
  0.07927077 0.11207601]
 [0.08685982 0.10274765 0.1165492  0.13861166 0.08700369 0.11593374
  0.15795189 0.18349783]
 [0.09175886 0.13810267 0.08606043 0.11435282 0.10001098 0.09824684
  0.14901404 0.13795896]
 [0.09805723 0.11559549 0.13281026 0.18622339 0.1525802  0.11022329
  0.16406138 0.0727964 ]
 [0.10331543 0.12159826 0.09394836 0.1037182  0.07208288 0.21389295
  0.08911909 0.07214747]
 [0.09964392 0.10164499 0.16238947 0.09800943 0.16946645 0.15731917
  0.13599394 0.1900072 ]
 [0.1908323  0.23116282 0.15534662 0.17054859 0.12621552 0.10012773
  0.1168507  0.09490333]]
